<a href="https://colab.research.google.com/github/shikhsh32/BigMart/blob/main/FoodDemandForcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving test_QoiMO9B.csv to test_QoiMO9B.csv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving fulfilment_center_info.csv to fulfilment_center_info.csv
Saving meal_info.csv to meal_info.csv
Saving train.csv to train.csv


In [3]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("FoodDemand").getOrCreate()


In [4]:
train = spark.read.csv("train.csv", header=True, inferSchema=True)
test = spark.read.csv("test_QoiMO9B.csv", header=True, inferSchema=True)
meal = spark.read.csv("meal_info.csv", header=True, inferSchema=True)
center = spark.read.csv("fulfilment_center_info.csv", header=True, inferSchema=True)

train.printSchema()
train.show(5)
train.describe().show()


root
 |-- id: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- center_id: integer (nullable = true)
 |-- meal_id: integer (nullable = true)
 |-- checkout_price: double (nullable = true)
 |-- base_price: double (nullable = true)
 |-- emailer_for_promotion: integer (nullable = true)
 |-- homepage_featured: integer (nullable = true)
 |-- num_orders: integer (nullable = true)

+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|     id|week|center_id|meal_id|checkout_price|base_price|emailer_for_promotion|homepage_featured|num_orders|
+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|1379560|   1|       55|   1885|        136.83|    152.29|                    0|                0|       177|
|1466964|   1|       55|   1993|        136.83|    135.83|                    0|                0|       270|
|1346989|   1|       55|   2539|        134.86|    135.

In [5]:
train.count(), test.count(), train.select("center_id").distinct().count()


(456548, 32573, 77)

In [9]:
# view Schema
train.printSchema(), test.printSchema(),meal.printSchema(),center.printSchema()

root
 |-- id: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- center_id: integer (nullable = true)
 |-- meal_id: integer (nullable = true)
 |-- checkout_price: double (nullable = true)
 |-- base_price: double (nullable = true)
 |-- emailer_for_promotion: integer (nullable = true)
 |-- homepage_featured: integer (nullable = true)
 |-- num_orders: integer (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- center_id: integer (nullable = true)
 |-- meal_id: integer (nullable = true)
 |-- checkout_price: double (nullable = true)
 |-- base_price: double (nullable = true)
 |-- emailer_for_promotion: integer (nullable = true)
 |-- homepage_featured: integer (nullable = true)

root
 |-- meal_id: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- cuisine: string (nullable = true)

root
 |-- center_id: integer (nullable = true)
 |-- city_code: integer (nullable = true)
 |-- region_code: integer (nullable = t

(None, None, None, None)

In [10]:
# check record count
print("Train rows:", train.count())
print("Test rows:", test.count())
print("Meal info rows:", meal.count())
print("Center info rows:", center.count())


Train rows: 456548
Test rows: 32573
Meal info rows: 51
Center info rows: 77


In [11]:
# Look at sample views
train.show(5)
meal.show(5)
center.show(5)
test.show(5)


+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|     id|week|center_id|meal_id|checkout_price|base_price|emailer_for_promotion|homepage_featured|num_orders|
+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|1379560|   1|       55|   1885|        136.83|    152.29|                    0|                0|       177|
|1466964|   1|       55|   1993|        136.83|    135.83|                    0|                0|       270|
|1346989|   1|       55|   2539|        134.86|    135.86|                    0|                0|       189|
|1338232|   1|       55|   2139|         339.5|    437.53|                    0|                0|        54|
|1448490|   1|       55|   2631|         243.5|     242.5|                    0|                0|        40|
+-------+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
only showi

In [15]:
#Check unique values
train.select("center_id").distinct().count()
train.select("meal_id").distinct().count()
train.select("week").distinct().count()


145

In [16]:
train.summary().show()

+-------+------------------+------------------+-----------------+------------------+------------------+------------------+---------------------+-------------------+------------------+
|summary|                id|              week|        center_id|           meal_id|    checkout_price|        base_price|emailer_for_promotion|  homepage_featured|        num_orders|
+-------+------------------+------------------+-----------------+------------------+------------------+------------------+---------------------+-------------------+------------------+
|  count|            456548|            456548|           456548|            456548|            456548|            456548|               456548|             456548|            456548|
|   mean|1250096.3056327046| 74.76877130115562|82.10579610468122|2024.3374584928638|332.23893255494767|354.15662745230014|  0.08115247465764827|0.10919990888143197| 261.8727603669275|
| stddev| 144354.8223778807|41.524956371246134|45.97504601530899| 547.4209201301

In [17]:
# unique center-meal Combination

train.select("center_id", "meal_id").distinct().count()


3597

In [18]:
# No. of weeks

train.select("week").distinct().orderBy("week").show()


+----+
|week|
+----+
|   1|
|   2|
|   3|
|   4|
|   5|
|   6|
|   7|
|   8|
|   9|
|  10|
|  11|
|  12|
|  13|
|  14|
|  15|
|  16|
|  17|
|  18|
|  19|
|  20|
+----+
only showing top 20 rows



In [19]:
# check missing values

from pyspark.sql.functions import col, sum

train.select([sum(col(c).isNull().cast("int")).alias(c) for c in train.columns]).show()


+---+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
| id|week|center_id|meal_id|checkout_price|base_price|emailer_for_promotion|homepage_featured|num_orders|
+---+----+---------+-------+--------------+----------+---------------------+-----------------+----------+
|  0|   0|        0|      0|             0|         0|                    0|                0|         0|
+---+----+---------+-------+--------------+----------+---------------------+-----------------+----------+



In [20]:
# Join data set into single master table
# train – contains historical demand & pricing fields

# meal – contains meal info (category, cuisine)

# center – contains center info (city, region, op_area)

df = train.join(meal, on="meal_id", how="left") \             # For every row in train, find the matching row in meal where meal.meal_id = train.meal_id
                                                              # Keep all rows from train even if the meal info is missing (because of how="left")
          .join(center, on="center_id", how="left")           # Take the merged dataset from step 1
                                                              # Add columns from center where center.center_id = train.center_id


In [22]:
df.printSchema()
df.show(5)
df.count()


root
 |-- center_id: integer (nullable = true)
 |-- meal_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- checkout_price: double (nullable = true)
 |-- base_price: double (nullable = true)
 |-- emailer_for_promotion: integer (nullable = true)
 |-- homepage_featured: integer (nullable = true)
 |-- num_orders: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- cuisine: string (nullable = true)
 |-- city_code: integer (nullable = true)
 |-- region_code: integer (nullable = true)
 |-- center_type: string (nullable = true)
 |-- op_area: double (nullable = true)

+---------+-------+-------+----+--------------+----------+---------------------+-----------------+----------+---------+-------+---------+-----------+-----------+-------+
|center_id|meal_id|     id|week|checkout_price|base_price|emailer_for_promotion|homepage_featured|num_orders| category|cuisine|city_code|region_code|center_type|op_area|
+---------+-------

456548